In [ ]:
from src import LaNoLem, plot_result
from src.Dysts_dataset import Data
import matplotlib.pyplot as plt
%matplotlib inline

Dataset = Data()
data_name='Halvorsen'
data = Dataset.make_each_data(data_name, noise_list=[0.5], seed_list=[19])[0.5][19]
setting = Dataset.make_setting(data_name)
model = LaNoLem(verbose=True)
model.random_state = 42
model = model.fit(data,fit_type='Robust', max_iter=50)
plot_result(model, data, setting, fsize=3.3)
plt.show()

In [ ]:
from src import LaNoLem, plot_result, make_data
import matplotlib.pyplot as plt
%matplotlib inline
data_name = 'outdoor'
data, setting = make_data.make_data(data_name)
model = LaNoLem(verbose=True, ptol=1e-10)
model.random_state = 42
model = model.fit(data, k = 3,
                  fit_type='Latent', max_iter=50)
plot_result(model, data, setting, fsize=3.3)